### Query OpenAlex with pagination for all people affiliated with an organization
This script queries the OpenAlex API '/authors' endpoint for all authors affiliated with an organization.
It takes a ROR URL as input which is used to filter all authors based on their 'last_known_institution.ror' value and outputs their respective ORCID.

In [1]:
# needed dependency to make HTTP calls
import requests

The input for the computation is a ROR ID or ROR URL.

In [2]:
# input parameter
example_ror="https://ror.org/021k10z87"

We use it to query OpenAlex at its '`/authors`' endpoint to retrieve all authors that specified the organization's ROR ID in the field '`last_known_institution.ror`'.

In [3]:
# OpenAlex endpoint to query for authors
OPENALEX_API_AUTHORS = "https://api.openalex.org/authors"

# query all people that are connected to given ROR
def download_data(ror):
    page = 1
    max_page = 1
    while page <= max_page:
        params = {'filter': 'last_known_institution.ror:'+ror, 'page': page}

        response = requests.get(url=OPENALEX_API_AUTHORS,
                                params=params,
                                headers= {'Content-Type': 'application/json'})
        result=response.json()

        # calculate max page number in first loop
        if max_page == 1:
          max_page = determine_max_page(result)
        page = page + 1
        yield result

# calculate max number of result pages
def determine_max_page(response_data):
    item_count = response_data['meta']['count']
    items_per_page = response_data['meta']['per_page']
    max_page_ceil = item_count // items_per_page + bool(item_count % items_per_page)
    return max_page_ceil


# example execution
list_of_pages=download_data(example_ror)
print(f"Found {len(list(list_of_pages))} page(s) for ROR {example_ror}")

Found 4 page(s) for ROR https://ror.org/021k10z87


From the result set we extract each person's metadata and output their respective ORCID and name:

In [4]:
# extract all ORCIDs from the result
def extract_orcids(data):
    for author in data['results']:
        if author['ids']['orcid']:
            orcid=author['ids']['orcid'].replace("https://orcid.org/", "")
            display_name=author['display_name']
            yield orcid, display_name

# example execution
list_of_pages=download_data(example_ror)
for page in list_of_pages:
    for orcid,name in extract_orcids(page):
        print(f"{orcid}, {name}")

0000-0002-3824-5375, Nicole Deitelhoff
0000-0002-7348-7206, Jonas Wolff
0000-0002-6891-770X, Francis O’Connor
0000-0002-3536-8898, Felix Anderl
0000-0002-4259-6071, Felix S. Bethke
0000-0002-3136-0901, Thorsten Gromes
0000-0001-9698-2616, Annika Elena Poppe
0000-0002-3783-6130, Irene Weipert-Fenner
0000-0002-4793-9010, Arvid Bell
0000-0002-7012-6739, Peter Kreuzer
0000-0002-0143-5183, Christina Kohler
